In [1]:
n = 30

weights = {
    1: 2,   2: 5,   3: 3,   4: 23,  5: 17,
    6: 8,   7: 12,  8: 1,   9: 4,  10: 5,
    11: 7,  12: 2,  13: 4,  14: 7,  15: 20,
    16: 5,  17: 22, 18: 7,  19: 4,  20: 1,
    21: 14, 22: 0,  23: 26, 24: 16, 25: 6,
    26: 15, 27: 8,  28: 4,  29: 8,  30: 4
}

durations = {
    1: 18,  2: 71,  3: 1,   4: 4,   5: 30,
    6: 97,  7: 73,  8: 83,  9: 34, 10: 22,
    11: 33, 12: 30, 13: 35, 14: 63, 15: 25,
    16: 84, 17: 87, 18: 83, 19: 46, 20: 63,
    21: 76, 22: 60, 23: 8,  24: 85, 25: 85,
    26: 26, 27: 83, 28: 86, 29: 36, 30: 82
}


ready_dates = {
    1: 4,     2: 28,    3: 85,    4: 69,    5: 373,
    6: 397,   7: 413,   8: 425,   9: 410,   10: 478,
    11: 696,  12: 768,  13: 812,  14: 902,  15: 903,
    16: 906,  17: 1012, 18: 1113, 19: 1163, 20: 1079,
    21: 1335, 22: 1294, 23: 1330, 24: 1497, 25: 1432,
    26: 1587, 27: 1338, 28: 1341, 29: 929,  30: 982
}


due_dates = {
    1: 83,    2: 619,   3: 649,   4: 724,   5: 1117,
    6: 565,   7: 1237,  8: 593,   9: 1001, 10: 677,
    11: 1370, 12: 1167, 13: 1348, 14: 1175, 15: 1000,
    16: 1212, 17: 1238, 18: 1990, 19: 1908, 20: 1323,
    21: 1915, 22: 2143, 23: 2037, 24: 2135, 25: 1981,
    26: 1870, 27: 2029, 28: 1641, 29: 1437, 30: 1084
}

deadlines = {
    1: 2201,  2: 1270,  3: 1213,  4: 2356,  5: 2392,
    6: 636,   7: 1773,  8: 1362,  9: 3116, 10: 2786,
    11: 2037, 12: 2393, 13: 3582, 14: 1627, 15: 2062,
    16: 3511, 17: 3599, 18: 4220, 19: 2048, 20: 2703,
    21: 2856, 22: 3939, 23: 3812, 24: 3881, 25: 3363,
    26: 3804, 27: 2312, 28: 3626, 29: 3426, 30: 2384
}


successors = {
    1: [2],
    2: [8, 7, 3],
    3: [11, 6],
    4: [8, 6],
    5: [6],
    6: [19, 12, 9],
    7: [19, 12, 9],
    8: [19, 9],
    9: [23, 17, 15, 14, 13],
    10: [22, 19, 17, 14, 13],
    11: [23, 19, 17, 13],
    12: [23, 16, 13],
    13: [21, 20, 18],
    14: [21, 20, 18],
    15: [22, 20, 18],
    16: [22, 21, 18],
    17: [21, 18],
    18: [28, 25, 24],
    19: [24, 21],
    20: [25, 24],
    21: [27, 25],
    22: [30, 24],
    23: [30, 24],
    24: [27, 26],
    25: [30, 26],
    26: [29],
    27: [29],
    28: [29],
    29: [],
    30: []
}



In [2]:
predecessors = {i: [] for i in range(1, n+1)}
for i in range(1, n+1):
    for s in successors[i]:
        predecessors[s].append(i)


In [3]:
from collections import deque

indeg = {i: len(predecessors[i]) for i in range(1, n+1)}
q = deque([i for i in range(1, n+1) if indeg[i] == 0])
topo = []

while q:
    u = q.popleft()
    topo.append(u)
    for v in successors[u]:
        indeg[v] -= 1
        if indeg[v] == 0:
            q.append(v)

if len(topo) != n:
    raise ValueError("Precedence graph has a cycle!")


In [4]:
new_ready_dates = {i: ready_dates[i] for i in range(1, n+1)}
for i in topo:
    if predecessors[i]:
        new_ready_dates[i] = max(ready_dates[i], max(new_ready_dates[j] + durations[j] for j in predecessors[i]))

In [5]:
new_deadlines = {i: deadlines[i] for i in range(1, n+1)}

for i in reversed(topo):
    if successors[i]:
        new_deadlines[i] = min(new_deadlines[i], min(new_deadlines[s] - durations[s] for s in successors[i]))

In [48]:
from pysat.formula import CNF, IDPool
from pysat.solvers import Glucose42
from pysat.card import CardEnc, EncType

def solve_with_prefix_precedence(n, durations, ready_dates, deadlines, successors):
    jobs = list(range(1, n+1))
    t_max = max(deadlines.values()) + 1

    vpool = IDPool()
    S = {}
    A = {}
    SC = {}
    valid = {}
    cnf = CNF()

    # 1) domains & vars
    for i in jobs:
        r_i = ready_dates[i] 
        p_i = durations[i]
        dl_i = deadlines[i]

        last_start = dl_i - p_i
        if last_start < r_i:
            print(f"Job {i} impossible: last_start < ready ({last_start} < {r_i})")
            return False, None
        valid[i] = list(range(r_i, last_start + 1))
        for t in valid[i]:
            S[(i,t)] = vpool.id(("S", i, t))
        for t in range(r_i, dl_i):
            A[(i,t)] = vpool.id(("A", i, t))

    # debug counts
    print("Vars summary: S = ", len(S), "A = ", len(A))

    # 2) exactly-one start per job (seqcounter)
    start_once_cls = 0
    for i in jobs:
        lits = [S[(i,t)] for t in valid[i]]
        enc = CardEnc.equals(lits=lits, bound=1, encoding=EncType.seqcounter)
        cnf.extend(enc.clauses)
        start_once_cls += len(enc.clauses)
    print("Start-once clauses: ", start_once_cls)

    # 3) activation S -> A
    s_to_a_cls = 0
    for i in jobs:
        p_i = durations[i]
        for t_start in valid[i]:
            lit_s = S[(i,t_start)]
            for t in range(t_start, t_start + p_i):
                if (i, t) in A:
                    cnf.append([-lit_s, A[(i,t)]])
                    s_to_a_cls += 1
    print("S->A clauses:", s_to_a_cls)

    # 4) capacity per time (seqcounter)
    cap_cls = 0
    for t in range(t_max):
        active_vars = [A[(i,t)] for i in jobs if (i,t) in A]
        if len(active_vars) > 1:
            enc = CardEnc.atmost(lits=active_vars, bound=1, encoding=EncType.seqcounter)
            cnf.extend(enc.clauses)
            cap_cls += len(enc.clauses)
    print("Capacity clauses:", cap_cls)

    # 5) build SC prefix variables and link both directions
    s_to_sc = 0
    sc_chain = 0
    sc_reverse = 0
    for j in jobs:
        if not valid[j]:
            continue
        min_t = min(valid[j])
        max_t = max(valid[j])
        # create SC vars on [min_t..max_t]
        for t in range(min_t, max_t + 1):
            SC[(j,t)] = vpool.id(("SC", j, t))
        cnf.append([SC[(j, max_t)]])
        # S -> SC (for same t)
        for t in valid[j]:
            cnf.append([-S[(j,t)], SC[(j,t)]])
            s_to_sc += 1
        # chain SC[t] -> SC[t+1]
        for t in range(min_t, max_t):
            cnf.append([-SC[(j,t)], SC[(j,t+1)]])
            sc_chain += 1
        # reverse: SC[t] -> OR_{u<=t} S[u]
        # build list of starts <= t
        for t in range(min_t, max_t + 1):
            sats = [S[(j,u)] for u in valid[j] if u <= t]
            # clause: -SC[j,t] ∨ S[j,u1] ∨ S[j,u2] ∨ ...
            if sats:
                clause = [-SC[(j,t)]] + sats
                cnf.append(clause)
                sc_reverse += 1
    print("SC vars:", len(SC))
    print("S->SC:", s_to_sc)
    print("SC chain:", sc_chain)
    print("SC reverse:", sc_reverse)

    # 6) precedence: S(i,t_i) -> not SC(j, finish_clamped)
    prec_cls = 0
    for i in jobs:
        for j in successors.get(i, []):
            if not valid[i] or not valid[j]:
                continue
            min_j = min(valid[j])
            max_j = max(valid[j])
            p_i = durations[i]
            for t_i in valid[i]:
                finish = t_i + p_i - 1
                if finish < min_j or finish > max_j:
                    continue
                # use the clamped key
                cnf.append([-S[(i,t_i)], -SC[(j, finish)]])
                prec_cls += 1
    print("Precedence clauses:", prec_cls)

    # 7) solve

    print("\n=== SOLVING (HARD CONSTRAINTS ONLY) ===")

    # Tạo solver
    solver = Glucose42()
    solver.append_formula(cnf)

    sat = solver.solve()

    if not sat:
        print("UNSAT — no feasible schedule.")
    else:
        model = solver.get_model()
        model_set = set(model)

        # Extract start times
        schedule = {}
        for (i, t), vid in S.items():
            if vid in model_set:
                schedule[i] = t

        # Nếu thiếu job nào (không được gán), in cảnh báo
        missing = [i for i in range(1, n+1) if i not in schedule]
        if missing:
            print("WARNING: Some jobs missing start times ->", missing)

        # Format output: sort by start time
        schedule_sorted = sorted(schedule.items(), key=lambda x: x[1])

        print("Feasible schedule found:\n")

        print("{:<8} {:<10} {:<10}".format("Job", "Start", "End"))
        print("-" * 30)

        for i, start in schedule_sorted:
            end = start + durations[i]
            print("{:<8} {:<10} {:<10}".format(i, start, end))

        print("\nJob order (by start time):", [i for i, _ in schedule_sorted])



In [50]:
solve_with_prefix_precedence(n, durations, ready_dates, deadlines, successors)

Vars summary: S =  53610 A =  55189
Start-once clauses:  160740
S->A clauses: 2741542
Capacity clauses: 148909
SC vars: 53610
S->SC: 53610
SC chain: 53580
SC reverse: 53610
Precedence clauses: 80960

=== SOLVING (HARD CONSTRAINTS ONLY) ===
UNSAT — no feasible schedule.
